In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from utils import *

import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_csv("../data/clean_data.csv", sep=",", index_col=0)
print(len(df))

9024


In [4]:
dois = list(df['doi'])
dois

['10.18653/v1/P18-1001',
 '10.18653/v1/P18-1002',
 '10.18653/v1/P18-1003',
 '10.18653/v1/P18-1004',
 '10.18653/v1/P18-1005',
 '10.18653/v1/P18-1006',
 '10.18653/v1/P18-1007',
 '10.18653/v1/P18-1008',
 '10.18653/v1/P18-1009',
 '10.18653/v1/P18-1010',
 '10.18653/v1/P18-1011',
 '10.18653/v1/P18-1012',
 '10.18653/v1/P18-1013',
 '10.18653/v1/P18-1014',
 '10.18653/v1/P18-1015',
 '10.18653/v1/P18-1016',
 '10.18653/v1/P18-1017',
 '10.18653/v1/P18-1018',
 '10.18653/v1/P18-1019',
 '10.18653/v1/P18-1020',
 '10.18653/v1/P18-1021',
 '10.18653/v1/P18-1022',
 '10.18653/v1/P18-1023',
 '10.1162/tacl_a_00026',
 '10.18653/v1/P18-1024',
 '10.18653/v1/P18-1025',
 '10.18653/v1/P18-1026',
 '10.18653/v1/P18-1027',
 '10.18653/v1/P18-1028',
 '10.18653/v1/P18-1029',
 '10.18653/v1/P18-1030',
 '10.18653/v1/P18-1031',
 '10.18653/v1/P18-1032',
 '10.18653/v1/P18-2001',
 '10.18653/v1/P18-2002',
 '10.18653/v1/P18-2003',
 '10.18653/v1/P18-2004',
 '10.18653/v1/P18-2005',
 '10.18653/v1/P18-2006',
 '10.18653/v1/W18-3407',


In [5]:
papers = bulk_get_paper_details(dois)
papers

 None,
 None,
 None,
 ...]

In [71]:
df['working_doi'] = [bool(paper) for paper in papers]
df

,id,title,area,interpretability,doi,source,working_doi
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018,True
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018,True
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018,True
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018,True
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018,True
...,...,...,...,...,...,...,...
9019,NaN,Neural Machine Translation with Monolingual Tr...,Machine Translation and Multilinguality,False,10.18653/v1/2021.acl-long.567,ACL2021,True
9020,NaN,Intrinsic Dimensionality Explains the Effectiv...,Interpretability and Analysis of Models for NLP,True,10.18653/v1/2021.acl-long.568,ACL2021,True
9021,NaN,UnNatural Language Inference,"Semantics: Sentence-level Semantics, Textual I...",False,10.18653/v1/2021.acl-long.569,ACL2021,True
9022,NaN,Including Signed Languages in Natural Language...,Theme,False,10.18653/v1/2021.acl-long.570,ACL2021,True


In [72]:
df['working_doi'].value_counts()

working_doi
True     9022
False       2
Name: count, dtype: int64

In [29]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def search_paper_by_title(title):
    base_url = 'https://api.semanticscholar.org/graph/v1/paper/search'
    response = requests.get(base_url, params={'query': title}, headers={"x-api-key": API_KEY})
    
    paper_dict = response.json()
    if 'data' not in paper_dict:
        return None
        
    for result in paper_dict['data']:
        if similar(title, result['title']) > 0.85:
            return result['paperId']
    


search_paper_by_title('Document Modeling with External Attention for Sentence Extraction')

'98346b7de76e905d9f23064cf02606e4771e03c4'

In [8]:
get_paper_details('10.18653/v1/P18-1188')

Exception: Error: 404 Not Found

In [42]:
from tqdm import tqdm
from time import sleep
tqdm.pandas()

def update_row(row):
    if not row['working_doi']:
        paper_id = search_paper_by_title(row['title'])
        if paper_id:
            row['doi'] = paper_id
            row['working_doi'] = True
    return row

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    updated_row = update_row(row)
    df.loc[index] = updated_row

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9024/9024 [00:15<00:00, 574.25it/s]


In [73]:
df[df['working_doi'] == False]

,id,title,area,interpretability,doi,source,working_doi
2334,1542,Understanding Back-Translation at Scale,Area G (Machine Translation and Multilingualit...,NaN,10.18653/v1/D18-1045,EMNLP2018,False
7641,1334,Social IQa: Commonsense Reasoning about Social...,Reasoning,NaN,10.18653/v1/D19-1454,EMNLP2019,False


In [66]:
# some papers could not be found just by searching semantic scholar

manual_corrections = {
    '10.18653/v1/P18-1203': 'arXiv:1801.06176',
    '10.18653/v1/2020.emnlp-main.248': 'arXiv:1909.13375',
    '10.18653/v1/2023.emnlp-main.713': 'arXiv:2301.12534',
    '10.18653/v1/2023.emnlp-main.44': 'arXiv:2305.14288',
    '10.18653/v1/2023.emnlp-main.651': '10.48550/arXiv.2212.09114',
    '10.18653/v1/2023.emnlp-main.755': '10.48550/arXiv.2310.17887',
    '10.18653/v1/2023.emnlp-main.827': '10.48550/arXiv.2311.16258',
    '10.18653/v1/2023.emnlp-main.1001': '10.48550/arXiv.2305.14237',
    '10.18653/v1/2023.acl-long.237': '10.48550/arXiv.2306.09802',
    '10.18653/v1/2021.acl-long.438': 'CorpusID:235683604',
    
}

for original_doi, updated_doi in manual_corrections.items():
    df.loc[df['doi'] == original_doi, 'doi'] = updated_doi

In [43]:
df['working_doi'].value_counts()

working_doi
True     9011
False      13
Name: count, dtype: int64

In [68]:
dois = list(df['doi'])
dois

['10.18653/v1/P18-1001',
 '10.18653/v1/P18-1002',
 '10.18653/v1/P18-1003',
 '10.18653/v1/P18-1004',
 '10.18653/v1/P18-1005',
 '10.18653/v1/P18-1006',
 '10.18653/v1/P18-1007',
 '10.18653/v1/P18-1008',
 '10.18653/v1/P18-1009',
 '10.18653/v1/P18-1010',
 '10.18653/v1/P18-1011',
 '10.18653/v1/P18-1012',
 '10.18653/v1/P18-1013',
 '10.18653/v1/P18-1014',
 '10.18653/v1/P18-1015',
 '10.18653/v1/P18-1016',
 '10.18653/v1/P18-1017',
 '10.18653/v1/P18-1018',
 '10.18653/v1/P18-1019',
 '10.18653/v1/P18-1020',
 '10.18653/v1/P18-1021',
 '10.18653/v1/P18-1022',
 '10.18653/v1/P18-1023',
 '10.1162/tacl_a_00026',
 '10.18653/v1/P18-1024',
 '10.18653/v1/P18-1025',
 '10.18653/v1/P18-1026',
 '10.18653/v1/P18-1027',
 '10.18653/v1/P18-1028',
 '10.18653/v1/P18-1029',
 '10.18653/v1/P18-1030',
 '10.18653/v1/P18-1031',
 '10.18653/v1/P18-1032',
 '10.18653/v1/P18-2001',
 '10.18653/v1/P18-2002',
 '10.18653/v1/P18-2003',
 '10.18653/v1/P18-2004',
 '10.18653/v1/P18-2005',
 '10.18653/v1/P18-2006',
 '10.18653/v1/W18-3407',


In [69]:
papers = bulk_get_paper_details(dois)
papers

 ...]

In [75]:
df.to_csv("../data/clean_data.csv")